## Char level language model

In [75]:
shakespeare_url = "http://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

In [118]:
import urllib.request
import os
import ssl
import numpy as np

ssl._create_default_https_context = ssl._create_unverified_context

os.makedirs('datasets/ch16-nlp', exist_ok = True)
urllib.request.urlretrieve(shakespeare_url, 'datasets/ch16-nlp/input.txt')

('datasets/ch16-nlp/input.txt', <http.client.HTTPMessage at 0x7fc73e462a50>)

In [133]:
with open('datasets/ch16-nlp/input.txt', 'r') as f:
    all_lines = f.read()
    
#     for ind in range(len(all_lines)):
#         all_lines[ind] = all_lines[ind].strip()

In [135]:
print(len(all_lines))

1115394


In [136]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [137]:
tokenizer = Tokenizer(char_level = True)

In [138]:
tokenizer.fit_on_texts(all_lines)

In [139]:
word_index = tokenizer.word_index
index_word = tokenizer.index_word

In [157]:
print(word_index.keys())
n_char = len(word_index)

dict_keys([' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd', 'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v', '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$'])


In [145]:
temp = tokenizer.texts_to_sequences([all_lines[:2]])

In [146]:
print(temp)

[[20, 6]]


In [147]:
train_texts = all_lines[:int(0.9*len(all_lines))]
valid_texts = all_lines[int(0.9*len(all_lines)):]

In [148]:
## Assume that we never see valid texts before
tokenizer.fit_on_texts(train_texts)

In [227]:
[train_sequences] = np.array(tokenizer.texts_to_sequences([train_texts])) - 1
[valid_sequences] = np.array(tokenizer.texts_to_sequences([valid_texts])) - 1

In [237]:
def preprocess(texts):
    [sequences] = np.array(tokenizer.texts_to_sequences([texts])) - 1
    return tf.one_hot(sequences, n_char)

In [239]:
preprocess('Hello')

<tf.Tensor: shape=(5, 39), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [166]:
maxlen = 50

In [212]:
print(train_sequences.shape)

(1003854,)


In [243]:
def prepare_dataset(sequences):
    
    trainset = tf.data.Dataset.from_tensor_slices(sequences)
    trainset = trainset.window(size = maxlen+1, shift = 1, drop_remainder = True)
    trainset = trainset.flat_map(lambda x : x.batch(maxlen+1))
    trainset = trainset.batch(32)
    trainset = trainset.map(lambda x : (x[:, :-1], x[:, 1:]))
    trainset = trainset.map(lambda train, target : (tf.one_hot(train, n_char), target))
    trainset = trainset.shuffle(100).prefetch(2)
    return trainset

In [244]:
trainset = prepare_dataset(train_sequences)
validset = prepare_dataset(valid_sequences)

In [245]:
for data in trainset.take(1):
    print(data)

(<tf.Tensor: shape=(32, 50, 39), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.,

## Create basic RNN model

In [246]:
keras.backend.clear_session()

model = keras.models.Sequential([
    keras.layers.Conv1D(32, 2, dilation_rate = 1, padding = 'causal', activation = 'elu', input_shape = [maxlen, n_char]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, 2, dilation_rate = 2, padding = 'causal', activation = 'elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, 2, dilation_rate = 4, padding = 'causal', activation = 'elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, 2, dilation_rate = 8, padding = 'causal', activation = 'elu'),
    keras.layers.BatchNormalization(),
    keras.layers.GRU(128, return_sequences = True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(n_char, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer = 'nadam')

In [247]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 50, 32)            2528      
_________________________________________________________________
batch_normalization (BatchNo (None, 50, 32)            128       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 48)            3120      
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 48)            192       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 64)            6208      
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 64)            256       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 50, 96)            1

In [320]:
# model.fit(trainset, epochs = 5, validation_data=validset)
## The model take too long to train

In [249]:
# generate sequences with naive method
def next_char(texts):
    next_char_class = model.predict_classes(preprocess(texts))[0][-1]
    return next_char_class

In [253]:
preprocess(['hello'])

<tf.Tensor: shape=(0, 39), dtype=float32, numpy=array([], shape=(0, 39), dtype=float32)>

In [260]:
def preprocess2(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, n_char)

def preprocess(texts):
    sequences = np.array(tokenizer.texts_to_sequences([texts])) - 1
    return tf.one_hot(sequences, n_char)

* batch with consecutive between batch

## Stateful RNN

In [401]:
def create_consecutive_batch(sequences):
    batch_size = 32
    window_size = maxlen + 1
    shift = window_size - 1
    
    sequences_parts = np.array_split(sequences, batch_size)
    datasets = []
    
    for part in sequences_parts:
        
        dataset = tf.data.Dataset.from_tensor_slices(part)
        dataset = dataset.window(window_size, shift, drop_remainder = True)
        dataset = dataset.flat_map(lambda x: x.batch(window_size))
        datasets.append(dataset)
        
    con_dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *window: tf.stack(window))
    con_dataset = con_dataset.map(lambda x : (x[:, :-1], x[:, 1:]))
    con_dataset = con_dataset.map(lambda train, target : (tf.one_hot(train, n_char), target)).prefetch(1)
    return con_dataset

In [402]:
train_set_stateful = create_consecutive_batch(train_sequences)
valid_set_stateful = create_consecutive_batch(valid_sequences)

In [403]:
for data in valid_set_stateful.take(2):
    print(data)

(<tf.Tensor: shape=(32, 50, 39), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0.,

In [404]:
keras.backend.clear_session()

model_stateful = keras.models.Sequential([
    keras.layers.Conv1D(32, 2, dilation_rate = 1, padding = 'causal', activation = 'elu', batch_input_shape = [32, maxlen, n_char]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, 2, dilation_rate = 2, padding = 'causal', activation = 'elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, 2, dilation_rate = 4, padding = 'causal', activation = 'elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, 2, dilation_rate = 8, padding = 'causal', activation = 'elu'),
    keras.layers.BatchNormalization(),
    keras.layers.GRU(128, return_sequences = True, dropout=0.2, 
                     recurrent_dropout=0.2, stateful=True),
    keras.layers.Dense(n_char, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer = 'nadam')

In [405]:
class ResetStateCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [406]:
model.fit(train_set_stateful, epochs = 5, validation_data=valid_set_stateful, callbacks = [ResetStateCallback()])

Epoch 1/5
627/627 [==============================] - 89s 141ms/step - loss: 2.0342 - acc: 0.4027 - val_loss: 1.9117 - val_acc: 0.4256
Epoch 2/5
627/627 [==============================] - 72s 115ms/step - loss: 1.7933 - acc: 0.4640 - val_loss: 1.8190 - val_acc: 0.4505
Epoch 3/5
627/627 [==============================] - 81s 130ms/step - loss: 1.7202 - acc: 0.4815 - val_loss: 1.7741 - val_acc: 0.4626
Epoch 4/5
627/627 [==============================] - 71s 113ms/step - loss: 1.6817 - acc: 0.4908 - val_loss: 1.7427 - val_acc: 0.4718
Epoch 5/5
627/627 [==============================] - 72s 114ms/step - loss: 1.6571 - acc: 0.4970 - val_loss: 1.7238 - val_acc: 0.4775


## Sentiment analysis

In [407]:
import tensorflow.keras as keras
import tensorflow as tf

In [411]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 1s 0us/step


In [413]:
print(x_train.shape)
print(x_train[0])

(25000,)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [414]:
word_index = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [426]:
index_word = {id_ + 3: word for word, id_ in word_index.items()}

for id_, word in enumerate(['<pad>', '<sos>', '<unk>']):
    index_word[id_] = word
    
" ".join([index_word[id_] for id_ in x_train[0]])

"<sos> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shou

In [428]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Shuffling and writing examples to /Users/boonchuay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXHZJOY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/boonchuay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXHZJOY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/boonchuay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXHZJOY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/boonchuay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [429]:
datasets.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [434]:
for data in datasets['train'].batch(2).take(1):
    print(data)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [480]:
def preprocess(x, y):
    x = tf.strings.substr(x, 0, 300)
    X_batch = tf.strings.regex_replace(x, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(x, b"[^a-zA-Z']", b" ")
    x = tf.strings.split(x)
    return x.to_tensor(default_value = '<pad>'), y

In [481]:
for data in datasets['train'].batch(2).map(preprocess).take(1):
    print(data)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie.',
        b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
        b'Walken', b'or', b'Michael', b'Ironside.', b'Both', b'are',
        b'great', b'actors,', b'but', b'this', b'must', b'simply', b'be',
        b'their', b'worst', b'role', b'in', b'history.', b'Even',
        b'their', b'great', b'acting', b'could', b'not', b'redeem',
        b'this', b"movie's", b'ridiculous', b'storyline.', b'This',
        b'movie', b'is', b'an', b'early', b'nineties', b'US',
        b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
       [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
        b'during', b'films,', b'but', b'this', b'is', b'usually', b'due',
        b'to', b'a', b'combination', b'of', b'things', b'including,',
        b'really', b'tired,', b'being', b'warm', b'and', b'comfortable',
        b'on', b'the', b'sette', b'and', b'having', b'just', 

In [482]:
from collections import Counter

counter = Counter()
for data in datasets['train'].batch(32).map(preprocess):
    for review in data[0]:
        counter.update(review.numpy())

**We will create tokenizer pipeline by ourselves!**

In [501]:
vocab_size = 10000
num_oov_bucket = 1000

my_word_ind = {id_: word[0] for id_, word in enumerate(counter.most_common(vocab_size))}
my_ind_word = {value: key for key, value in my_word_ind.items()}

In [503]:
words = [word[0] for id_, word in enumerate(counter.most_common(vocab_size))]
indices = tf.range(len(words), dtype = tf.int64)

In [504]:
table = tf.lookup.KeyValueTensorInitializer(words, indices)
table = tf.lookup.StaticVocabularyTable(table, num_oov_bucket)

In [510]:
table.lookup(tf.constant('sadrussia'))

<tf.Tensor: shape=(), dtype=int64, numpy=10449>

In [534]:
def tokenize(string_tensor):
    return table.lookup(string_tensor)
def tokenize_xybatch(string_tensor, target):
    return table.lookup(string_tensor), target

## Create simple model

In [523]:
keras.backend.clear_session()

model = keras.models.Sequential([
    keras.layers.Lambda(lambda x : tokenize(x), input_shape = [None], dtype=tf.string),
    keras.layers.Embedding(vocab_size+num_oov_bucket, 50, mask_zero = True),
    keras.layers.Conv1D(32, 3, dilation_rate = 1, activation = 'relu', padding='causal'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, 3, dilation_rate = 2, activation = 'relu', padding='causal'),
    keras.layers.BatchNormalization(),   
    keras.layers.Conv1D(64, 3, dilation_rate = 4, activation = 'relu', padding='causal'),
    keras.layers.BatchNormalization(), 
    keras.layers.Conv1D(80, 3, dilation_rate = 8, activation = 'relu', padding='causal'),
    keras.layers.BatchNormalization(),
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout = 0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'nadam', metrics=['accuracy'])

In [524]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          550000    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 32)          4832      
_________________________________________________________________
batch_normalization (BatchNo (None, None, 32)          128       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 48)          4656      
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 48)          192       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          9

In [517]:
for data in datasets['train'].batch(2).map(preprocess).take(1):
    print(tf.shape(data[0]))

tf.Tensor([ 2 53], shape=(2,), dtype=int32)


In [527]:
model.fit(datasets['train'].batch(32).map(preprocess), validation_data=datasets['test'].batch(32).map(preprocess))

782/782 [==============================] - 158s 202ms/step - loss: 0.6611 - accuracy: 0.5901 - val_loss: 0.6198 - val_accuracy: 0.6455


In [528]:
import tensorflow_hub as hub

In [529]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, "datasets/ch16-nlp/my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [530]:
keras.backend.clear_session()

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [570]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

./datasets/ch16-nlp/my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
./datasets/ch16-nlp/my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/saved_model.pb
./datasets/ch16-nlp/my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.data-00000-of-00001
./datasets/ch16-nlp/my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/variables/variables.index
./datasets/ch16-nlp/my_tfhub_cache/82c4aaf4250ffb09088bd48368ee7fd00e5464fe/assets/tokens.txt


In [571]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 48,197,257
Trainable params: 6,657
Non-trainable params: 48,190,600
_________________________________________________________________


**Model in tensorhub is created with TF2.0, which incompatible with TF2.3!**

import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5)


### Automatic Translation

In [580]:
keras.backend.clear_session()

import tensorflow_addons as tfa


encoder_inputs = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=tf.int32)
sequence_length_inputs = keras.layers.Input(shape=[], dtype=tf.int32)

embedding_layer = keras.layers.Embedding(vocab_size, 128)
encoder_embeddings = embedding_layer(encoder_inputs)
decoder_embeddings = embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(128, return_state = True)
encoder_output, state_h, state_c = encoder(encoder_embeddings)

decoder_cell = keras.layers.LSTMCell(128)
sampler = tfa.seq2seq.TrainingSampler()
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(cell=decoder_cell, sampler = sampler,
                                                output_layer = output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(decoder_embeddings, 
                         initial_state = [state_h, state_c], 
                         sequence_length=sequence_length_inputs)

Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs, sequence_length_inputs], outputs=Y_proba)

In [611]:
model.summary()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 128)    1280000     input_22[0][0]                   
                                                                 input_23[0][0]                   
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, 128), (None, 131584      embedding_7[0][0]     

In [612]:
x = np.random.randint(vocab_size, size=(1000, 20))
y = np.random.randint(vocab_size, size=(1000, 30))
seq_len = np.full([1000], 30)

y_decoder = np.c_[np.zeros((1000)), y[:, :-1]]

In [613]:
print(x.shape)
print(y.shape)
print(seq_len.shape)
print(y_decoder.shape)

(1000, 20)
(1000, 30)
(1000,)
(1000, 30)


In [614]:
model.fit([x, y_decoder, seq_len],y)

32/32 [==============================] - 13s 402ms/step - loss: 9.2103 - acc: 1.0000e-04
